# Group-level comparison across confidence SNR

This notebook simulates 20 participants (100 trials each) across 5 levels of type-2 noise
(lower noise = higher confidence SNR) and compares four group-level approaches:

- MLE per-subject mean
- MLE pooled counts (per condition)
- Pooled Bayesian (per condition)
- Hierarchical Bayesian (rm1way, condition offsets)


In [1]:
import os
os.environ.setdefault("PYTENSOR_FLAGS", "compiledir=/tmp/pytensor")
os.environ.setdefault("MPLCONFIGDIR", "/tmp/matplotlib")

import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt

from metadpy.utils import type2_SDT_simuation, ratings2df, responseSimulation, trials2counts
from metadpy.mle import metad
from metadpy.bayesian import hmetad, hmetad_pooled


In [2]:
np.random.seed(123)

n_subjects = 20
n_trials = 500
n_ratings = 4
d = 1.0
c = 0.0

noise_levels = np.array([1.0, 0.8, 0.6, 0.4, 0.2])
snr_levels = 1.0 / noise_levels

num_samples = 400
num_chains = 2
tune = 400
target_accept = 0.9
random_seed = 123

calib_trials = 200000
mc_reps = 200


In [3]:
n_levels = len(noise_levels)
nR_S1_all = np.zeros((n_subjects, n_levels, 2 * n_ratings))
nR_S2_all = np.zeros_like(nR_S1_all)
frames = []

for subj in range(n_subjects):
    for idx, noise in enumerate(noise_levels):
        nR_S1, nR_S2 = type2_SDT_simuation(
            d=d,
            noise=float(noise),
            c=c,
            nRatings=n_ratings,
            nTrials=n_trials,
        )
        nR_S1 = nR_S1.astype(int)
        nR_S2 = nR_S2.astype(int)
        nR_S1_all[subj, idx, :] = nR_S1
        nR_S2_all[subj, idx, :] = nR_S2
        df = ratings2df(nR_S1, nR_S2)
        df["Subject"] = subj
        df["Condition"] = snr_levels[idx]
        frames.append(df)

sim_df = pd.concat(frames, ignore_index=True)
sim_df.head()


In [4]:
mle_df = metad(
    data=sim_df,
    nRatings=n_ratings,
    stimuli="Stimuli",
    accuracy="Accuracy",
    confidence="Confidence",
    subject="Subject",
    within="Condition",
    padding=False,
    verbose=0,
)

mle_group = (
    mle_df.groupby("Condition")["m_ratio"]
    .agg(mean="mean", sem=lambda x: x.std(ddof=1) / np.sqrt(x.count()))
    .reset_index()
)
mle_group["lower"] = mle_group["mean"] - 1.96 * mle_group["sem"]
mle_group["upper"] = mle_group["mean"] + 1.96 * mle_group["sem"]
mle_group["method"] = "MLE (per-subject mean)"


/Users/yifei/Documents/work/python/metacognition/metadpy/metadpy/mle.py:529: UserWarning: Your inputs contain zeros and is not corrected.This may interfere with proper estimation of meta-d.See docstrings for more information.
  warnings.warn(
/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:685: RuntimeWarning: invalid value encountered in scalar subtract
  dx = [(x0[i] + h[i]) - x0[i] for i in range(n)]
/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py:800: RuntimeWarning: invalid value encountered in dot
  self.f = self.J.dot(self.x)
/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/scipy/optimize/_trustregion_constr/tr_interior_point.py:114: Runtime

In [5]:
pooled_rows = []
for idx, snr in enumerate(snr_levels):
    nR_S1_pool = nR_S1_all[:, idx, :].sum(axis=0)
    nR_S2_pool = nR_S2_all[:, idx, :].sum(axis=0)
    res = metad(
        nR_S1=nR_S1_pool,
        nR_S2=nR_S2_pool,
        nRatings=n_ratings,
        padding=False,
        verbose=0,
    )
    pooled_rows.append(
        {
            "Condition": float(snr),
            "mean": float(res["m_ratio"].iloc[0]),
            "lower": float(res["m_ratio"].iloc[0]),
            "upper": float(res["m_ratio"].iloc[0]),
            "method": "MLE (pooled counts)",
        }
    )

pooled_mle = pd.DataFrame(pooled_rows)


In [6]:
def summarize_posterior(da, hdi_prob=0.94):
    if hasattr(da, "to_array"):
        da = da.to_array()
    mean = da.mean(dim=("chain", "draw")).values
    hdi = az.hdi(da, hdi_prob=hdi_prob)
    if hasattr(hdi, "to_array"):
        hdi = hdi.to_array()
    lower = hdi.sel(hdi="lower").values
    upper = hdi.sel(hdi="higher").values
    return np.squeeze(mean), np.squeeze(lower), np.squeeze(upper)

pooled_models = hmetad_pooled(
    data=sim_df,
    nRatings=n_ratings,
    stimuli="Stimuli",
    accuracy="Accuracy",
    confidence="Confidence",
    within="Condition",
    sample_model=True,
    num_samples=num_samples,
    num_chains=num_chains,
    tune=tune,
    target_accept=target_accept,
    random_seed=random_seed,
    progressbar=False,
    output="model",
)

pooled_bayes_rows = []
for cond, (model, trace) in pooled_models.items():
    mratio_post = trace.posterior["meta_d"] / trace.posterior["d1"]
    mean, lower, upper = summarize_posterior(mratio_post)
    pooled_bayes_rows.append(
        {
            "Condition": float(cond),
            "mean": float(mean),
            "lower": float(lower),
            "upper": float(upper),
            "method": "Bayes (pooled)",
        }
    )

pooled_bayes = pd.DataFrame(pooled_bayes_rows)


Initializing NUTS using jitter+adapt_diag...
ERROR (pytensor.graph.rewriting.basic): SequentialGraphRewriter apply <pytensor.tensor.rewriting.elemwise.FusionOptimizer object at 0x11db08950>
ERROR (pytensor.graph.rewriting.basic): Traceback:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 289, in apply
    sub_prof = rewriter.apply(fgraph)
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/pytensor/tensor/rewriting/elemwise.py", line 886, in apply
    scalar_inputs, scalar_outputs = self.elemwise_to_scalar(inputs, outputs)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yifei/anaconda3/envs/metadpy-dev/lib/python3.11/site-packages/pytensor/tensor/rewriting/elemwise.py", line 538, in elemwise_to_scalar
    scalar_inputs = [replacement[inp]

In [7]:
hb_model, hb_trace = hmetad(
    data=sim_df,
    nRatings=n_ratings,
    stimuli="Stimuli",
    accuracy="Accuracy",
    confidence="Confidence",
    subject="Subject",
    within="Condition",
    sample_model=True,
    num_samples=num_samples,
    num_chains=num_chains,
    tune=tune,
    target_accept=target_accept,
    random_seed=random_seed,
    progressbar=False,
)

group_meta = hb_trace.posterior["mu_meta_d"] + hb_trace.posterior["condition_offset"]
group_mratio = group_meta / hb_trace.posterior["mu_d1"]
hb_mean, hb_lower, hb_upper = summarize_posterior(group_mratio)

hb_rows = []
for cond, mean, lower, upper in zip(snr_levels, hb_mean, hb_lower, hb_upper):
    hb_rows.append(
        {
            "Condition": float(cond),
            "mean": float(mean),
            "lower": float(lower),
            "upper": float(upper),
            "method": "Bayes (hierarchical)",
        }
    )

hb_summary = pd.DataFrame(hb_rows)


                                                                                
                              Step      Grad      Sampli…                       
  Progre…   Draws   Diverg…   size      evals     Speed     Elapsed   Remaini…  
 ────────────────────────────────────────────────────────────────────────────── 
  ━━━━━━━   1400    400       0.007     3         19.81     0:01:10   0:00:00   
                                                  draws/s                       
  ━━━━━━━   1400    1         0.022     255       9.83      0:02:22   0:00:00   
                                                  draws/s                       
                                                                                


In [8]:
summary = pd.concat([mle_group, pooled_mle, pooled_bayes, hb_summary], ignore_index=True)
summary = summary.sort_values(["method", "Condition"])

fig, ax = plt.subplots(figsize=(8, 4))
methods = ["MLE (per-subject mean)", "MLE (pooled counts)", "Bayes (pooled)", "Bayes (hierarchical)"]
markers = {
    "MLE (per-subject mean)": "o",
    "MLE (pooled counts)": "s",
    "Bayes (pooled)": "D",
    "Bayes (hierarchical)": "^",
}

for method in methods:
    sub = summary[summary["method"] == method].sort_values("Condition")
    yerr = np.vstack([sub["mean"] - sub["lower"], sub["upper"] - sub["mean"]])
    ax.errorbar(
        sub["Condition"],
        sub["mean"],
        yerr=yerr,
        label=method,
        marker=markers[method],
        capsize=3,
    )

ax.set_xlabel("Confidence SNR (1 / noise)")
ax.set_ylabel("Group m_ratio")
ax.set_title("Group-level meta-d'/d' across type-2 noise levels")
ax.legend(frameon=False)
fig.savefig("notebooks/group_level_methods_comparison.png", dpi=150, bbox_inches="tight")
plt.show()


<string>:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


In [9]:
effective_mratio = []
for noise in noise_levels:
    nR_S1_truth, nR_S2_truth = type2_SDT_simuation(
        d=d,
        noise=float(noise),
        c=c,
        nRatings=n_ratings,
        nTrials=calib_trials,
    )
    nR_S1_truth = nR_S1_truth.astype(int)
    nR_S2_truth = nR_S2_truth.astype(int)
    calib_res = metad(
        nR_S1=nR_S1_truth,
        nR_S2=nR_S2_truth,
        nRatings=n_ratings,
        padding=True,
        verbose=0,
    )
    effective_mratio.append(float(calib_res["m_ratio"].iloc[0]))

effective_mratio = np.array(effective_mratio)

truth_rows = []
for snr, mratio in zip(snr_levels, effective_mratio):
    mc_vals = []
    for rep in range(mc_reps):
        sim_truth = responseSimulation(
            d=d,
            metad=mratio * d,
            c=c,
            nRatings=n_ratings,
            nTrials=n_trials,
            nSubjects=n_subjects,
        )
        nR_S1_pool, nR_S2_pool = trials2counts(
            data=sim_truth,
            stimuli="Stimuli",
            accuracy="Accuracy",
            confidence="Confidence",
            nRatings=n_ratings,
            padding=True,
        )
        res = metad(
            nR_S1=nR_S1_pool,
            nR_S2=nR_S2_pool,
            nRatings=n_ratings,
            padding=True,
            verbose=0,
        )
        val = float(res["m_ratio"].iloc[0])
        if np.isfinite(val):
            mc_vals.append(val)
    truth_rows.append(
        {
            "Condition": float(snr),
            "truth_mratio": float(np.mean(mc_vals)),
        }
    )

truth_df = pd.DataFrame(truth_rows)
summary_with_truth = summary.merge(truth_df, on="Condition", how="left")
summary_with_truth["diff"] = summary_with_truth["mean"] - summary_with_truth["truth_mratio"]
summary_with_truth["diff_lower"] = summary_with_truth["lower"] - summary_with_truth["truth_mratio"]
summary_with_truth["diff_upper"] = summary_with_truth["upper"] - summary_with_truth["truth_mratio"]
summary_with_truth


In [10]:
fig, ax = plt.subplots(figsize=(8, 4))
for method in methods:
    sub = summary_with_truth[summary_with_truth["method"] == method].sort_values("Condition")
    yerr = np.vstack([sub["diff"] - sub["diff_lower"], sub["diff_upper"] - sub["diff"]])
    ax.errorbar(
        sub["Condition"],
        sub["diff"],
        yerr=yerr,
        label=method,
        marker=markers[method],
        capsize=3,
    )

ax.axhline(0, color="black", linewidth=1, alpha=0.5)
ax.set_xlabel("Confidence SNR (1 / noise)")
ax.set_ylabel("Estimate - ground truth (m_ratio)")
ax.set_title("Bias relative to ground truth across type-2 noise levels")
ax.legend(frameon=False)
fig.savefig("notebooks/group_level_methods_comparison_bias.png", dpi=150, bbox_inches="tight")
plt.show()


<string>:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


## Notes
- Lower noise (higher SNR) should yield higher estimated m_ratio across methods.
- The hierarchical model typically provides more stable group-level trends
  when per-subject estimates are noisy (low SNR / few trials).
- Sampling settings are intentionally light for speed; increase draws for final runs.


## Low-trial scenario (fewer trials per subject)

This section repeats the comparison with fewer trials per subject to highlight
the advantage of hierarchical Bayesian shrinkage under noisy individual estimates.


In [11]:
n_trials_low = 100
n_subjects_low = n_subjects
padding_low = True

n_levels = len(noise_levels)
nR_S1_low = np.zeros((n_subjects_low, n_levels, 2 * n_ratings))
nR_S2_low = np.zeros_like(nR_S1_low)
frames_low = []

for subj in range(n_subjects_low):
    for idx, noise in enumerate(noise_levels):
        nR_S1, nR_S2 = type2_SDT_simuation(
            d=d,
            noise=float(noise),
            c=c,
            nRatings=n_ratings,
            nTrials=n_trials_low,
        )
        nR_S1 = nR_S1.astype(int)
        nR_S2 = nR_S2.astype(int)
        nR_S1_low[subj, idx, :] = nR_S1
        nR_S2_low[subj, idx, :] = nR_S2
        df = ratings2df(nR_S1, nR_S2)
        df["Subject"] = subj
        df["Condition"] = snr_levels[idx]
        frames_low.append(df)

sim_df_low = pd.concat(frames_low, ignore_index=True)
sim_df_low.head()


In [12]:
mle_rows_low = []
subjects_low = pd.unique(sim_df_low["Subject"])
conditions_low = pd.unique(sim_df_low["Condition"])
for sub in subjects_low:
    for cond in conditions_low:
        subset = sim_df_low[(sim_df_low["Subject"] == sub) & (sim_df_low["Condition"] == cond)]
        nR_S1_sub, nR_S2_sub = trials2counts(
            data=subset,
            stimuli="Stimuli",
            accuracy="Accuracy",
            confidence="Confidence",
            nRatings=n_ratings,
            padding=padding_low,
        )
        try:
            res = metad(
                nR_S1=nR_S1_sub,
                nR_S2=nR_S2_sub,
                nRatings=n_ratings,
                padding=False,
                verbose=0,
            )
            res = res.assign(Subject=sub, Condition=cond)
        except Exception:
            res = pd.DataFrame(
                {
                    "dprime": [np.nan],
                    "meta_d": [np.nan],
                    "m_ratio": [np.nan],
                    "m_diff": [np.nan],
                    "Subject": [sub],
                    "Condition": [cond],
                }
            )
        mle_rows_low.append(res)

mle_df_low = pd.concat(mle_rows_low, ignore_index=True)

mle_group_low = (
    mle_df_low.groupby("Condition")["m_ratio"]
    .agg(mean="mean", sem=lambda x: x.std(ddof=1) / np.sqrt(x.count()))
    .reset_index()
)
mle_group_low["lower"] = mle_group_low["mean"] - 1.96 * mle_group_low["sem"]
mle_group_low["upper"] = mle_group_low["mean"] + 1.96 * mle_group_low["sem"]
mle_group_low["method"] = "MLE (per-subject mean)"


In [13]:
pooled_rows_low = []
for idx, snr in enumerate(snr_levels):
    nR_S1_pool = nR_S1_low[:, idx, :].sum(axis=0)
    nR_S2_pool = nR_S2_low[:, idx, :].sum(axis=0)
    try:
        res = metad(
            nR_S1=nR_S1_pool,
            nR_S2=nR_S2_pool,
            nRatings=n_ratings,
            padding=padding_low,
            verbose=0,
        )
        mean_val = float(res["m_ratio"].iloc[0])
    except Exception:
        mean_val = np.nan
    pooled_rows_low.append(
        {
            "Condition": float(snr),
            "mean": mean_val,
            "lower": mean_val,
            "upper": mean_val,
            "method": "MLE (pooled counts)",
        }
    )

pooled_mle_low = pd.DataFrame(pooled_rows_low)


In [14]:
initvals_low = {
    "c1": 0.0,
    "d1": 1.0,
    "meta_d": 1.0,
    "cS1_hn": np.full(n_ratings - 1, 0.5),
    "cS2_hn": np.full(n_ratings - 1, 0.5),
}

try:
    pooled_models_low = hmetad_pooled(
        data=sim_df_low,
        nRatings=n_ratings,
        stimuli="Stimuli",
        accuracy="Accuracy",
        confidence="Confidence",
        within="Condition",
        padding=padding_low,
        sample_model=True,
        num_samples=num_samples,
        num_chains=1,
        tune=tune,
        target_accept=target_accept,
        random_seed=random_seed,
        progressbar=False,
        output="model",
        init="adapt_diag",
        initvals=initvals_low,
        jitter_max_retries=0,
        cores=1,
    )

    pooled_bayes_rows_low = []
    for cond, (model, trace) in pooled_models_low.items():
        mratio_post = trace.posterior["meta_d"] / trace.posterior["d1"]
        mean, lower, upper = summarize_posterior(mratio_post)
        pooled_bayes_rows_low.append(
            {
                "Condition": float(cond),
                "mean": float(mean),
                "lower": float(lower),
                "upper": float(upper),
                "method": "Bayes (pooled)",
            }
        )

    pooled_bayes_low = pd.DataFrame(pooled_bayes_rows_low)
except Exception:
    pooled_bayes_low = pd.DataFrame(
        {
            "Condition": snr_levels.astype(float),
            "mean": np.nan,
            "lower": np.nan,
            "upper": np.nan,
            "method": "Bayes (pooled)",
        }
    )


In [15]:
hb_model_low, hb_trace_low = hmetad(
    data=sim_df_low,
    nRatings=n_ratings,
    stimuli="Stimuli",
    accuracy="Accuracy",
    confidence="Confidence",
    subject="Subject",
    within="Condition",
    padding=padding_low,
    sample_model=True,
    num_samples=num_samples,
    num_chains=num_chains,
    tune=tune,
    target_accept=target_accept,
    random_seed=random_seed,
    progressbar=False,
)

group_meta_low = hb_trace_low.posterior["mu_meta_d"] + hb_trace_low.posterior["condition_offset"]
group_mratio_low = group_meta_low / hb_trace_low.posterior["mu_d1"]
hb_mean_low, hb_lower_low, hb_upper_low = summarize_posterior(group_mratio_low)

hb_rows_low = []
for cond, mean, lower, upper in zip(snr_levels, hb_mean_low, hb_lower_low, hb_upper_low):
    hb_rows_low.append(
        {
            "Condition": float(cond),
            "mean": float(mean),
            "lower": float(lower),
            "upper": float(upper),
            "method": "Bayes (hierarchical)",
        }
    )

hb_summary_low = pd.DataFrame(hb_rows_low)


SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'mu_d1': array(-0.12020242), 'sigma_d1_log__': array(0.10291319), 'd1': array([[-0.42676494, -0.47243314, -0.2790231 , -0.10287953, -0.4502611 ],
       [-0.18607105, -0.77220957, -0.11944165, -0.17509695,  0.83033442],
       [-0.12229742, -0.51434947, -1.03117288,  0.72560764,  0.85613197],
       [-0.16940658, -0.0641974 , -0.70056302,  0.39876904, -0.10460456],
       [-0.69952706,  0.74035956,  0.18874597, -0.34404821,  0.62120573],
       [ 0.11523019,  0.54782616, -0.54868389, -0.19100467, -0.19460093],
       [-1.0952814 ,  0.35572742, -0.25001411,  0.27746568, -0.96097275],
       [-0.89380012, -0.29982289, -0.3535083 , -0.07675784, -0.96379132],
       [ 0.86903144,  0.10349025,  0.35848654, -0.8487804 ,  0.00193005],
       [-0.39389313, -0.62123198,  0.19445306, -0.32966312, -0.61925221],
       [ 0.81567693,  0.60100236, -0.54218276, -0.31493611,  0.28382398],
       [-0.07558377, -0.920

In [ ]:
summary_low = pd.concat([mle_group_low, pooled_mle_low, pooled_bayes_low, hb_summary_low], ignore_index=True)
summary_low = summary_low.sort_values(["method", "Condition"])

fig, ax = plt.subplots(figsize=(8, 4))
for method in methods:
    sub = summary_low[summary_low["method"] == method].sort_values("Condition")
    yerr = np.vstack([sub["mean"] - sub["lower"], sub["upper"] - sub["mean"]])
    ax.errorbar(
        sub["Condition"],
        sub["mean"],
        yerr=yerr,
        label=method,
        marker=markers[method],
        capsize=3,
    )

ax.set_xlabel("Confidence SNR (1 / noise)")
ax.set_ylabel("Group m_ratio")
ax.set_title("Group-level meta-d'/d' with low trials per subject")
ax.legend(frameon=False)
fig.savefig("notebooks/group_level_methods_comparison_low_trials.png", dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
truth_rows_low = []
for snr, mratio in zip(snr_levels, effective_mratio):
    mc_vals = []
    for rep in range(mc_reps):
        sim_truth = responseSimulation(
            d=d,
            metad=mratio * d,
            c=c,
            nRatings=n_ratings,
            nTrials=n_trials_low,
            nSubjects=n_subjects_low,
        )
        nR_S1_pool, nR_S2_pool = trials2counts(
            data=sim_truth,
            stimuli="Stimuli",
            accuracy="Accuracy",
            confidence="Confidence",
            nRatings=n_ratings,
            padding=True,
        )
        try:
            res = metad(
                nR_S1=nR_S1_pool,
                nR_S2=nR_S2_pool,
                nRatings=n_ratings,
                padding=True,
                verbose=0,
            )
            val = float(res["m_ratio"].iloc[0])
        except Exception:
            val = np.nan
        if np.isfinite(val):
            mc_vals.append(val)
    truth_rows_low.append(
        {
            "Condition": float(snr),
            "truth_mratio": float(np.mean(mc_vals)) if mc_vals else np.nan,
        }
    )

truth_df_low = pd.DataFrame(truth_rows_low)
summary_with_truth_low = summary_low.merge(truth_df_low, on="Condition", how="left")
summary_with_truth_low["diff"] = summary_with_truth_low["mean"] - summary_with_truth_low["truth_mratio"]
summary_with_truth_low["diff_lower"] = summary_with_truth_low["lower"] - summary_with_truth_low["truth_mratio"]
summary_with_truth_low["diff_upper"] = summary_with_truth_low["upper"] - summary_with_truth_low["truth_mratio"]
summary_with_truth_low


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for method in methods:
    sub = summary_with_truth_low[summary_with_truth_low["method"] == method].sort_values("Condition")
    yerr = np.vstack([sub["diff"] - sub["diff_lower"], sub["diff_upper"] - sub["diff"]])
    ax.errorbar(
        sub["Condition"],
        sub["diff"],
        yerr=yerr,
        label=method,
        marker=markers[method],
        capsize=3,
    )

ax.axhline(0, color="black", linewidth=1, alpha=0.5)
ax.set_xlabel("Confidence SNR (1 / noise)")
ax.set_ylabel("Estimate - ground truth (m_ratio)")
ax.set_title("Bias relative to ground truth (low trials)")
ax.legend(frameon=False)
fig.savefig("notebooks/group_level_methods_comparison_bias_low_trials.png", dpi=150, bbox_inches="tight")
plt.show()
